In [21]:
!pip3 install pickle5
!pip install --upgrade pandas
!pip install -U kiwipiepy gensim kss -q

     |████████████████████████████████| 502 kB 10.2 MB/s 
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
     |████████████████████████████████| 42.4 MB 135 kB/s 
     |████████████████████████████████| 25.7 MB 1.4 MB/s 
     |████████████████████████████████| 170 kB 35.6 MB/s 


In [22]:
import pandas as pd
import pickle5 as pickle
import re
import regex
from tqdm import tqdm

from kiwipiepy import Kiwi, Option
kiwi = Kiwi()
kiwi.prepare()

from kss import split_sentences
import itertools
from collections import defaultdict

In [4]:
#https://drive.google.com/file/d/1Oh32yF7mR_q90ybwAFM6iCAlrFq7UCj9/view?usp=sharing
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1Oh32yF7mR_q90ybwAFM6iCAlrFq7UCj9" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1Oh32yF7mR_q90ybwAFM6iCAlrFq7UCj9" -o df_trans.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2009      0 --:--:-- --:--:-- --:--:--  2009
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  430M  100  430M    0     0  92.7M      0  0:00:04  0:00:04 --:--:--  112M


In [5]:
with open('df_trans.pkl', "rb") as fh:
  data = pickle.load(fh)

In [6]:
#총 105624 개의 문단(페이지)
data

,ID,원문,번역문
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)..."
...,...,...,...
105619,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)..."
105620,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋..."
105621,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것..."
105622,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格..."


In [7]:
#번역문 크롤링 실패한 목록
data[data['번역문'] =='번역문 오류']

,ID,원문,번역문
2976,ITKC_MO_0011A_0170_010_0020,원문 오류,번역문 오류
12002,ITKC_MO_0049A_0010_010_0040,원문 오류,번역문 오류
12003,ITKC_MO_0049A_0010_010_0050,원문 오류,번역문 오류
27214,ITKC_MO_0189A_0160_010_0010,원문 오류,번역문 오류
27215,ITKC_MO_0189A_0160_010_0020,원문 오류,번역문 오류
...,...,...,...
37540,ITKC_MO_0273A_0360_010_0520,원문 오류,번역문 오류
37541,ITKC_MO_0273A_0360_010_0530,원문 오류,번역문 오류
46200,ITKC_MO_0299A_0130_010_0010,원문 오류,번역문 오류
47243,ITKC_MO_0311A_0070_010_0490,원문 오류,번역문 오류


In [8]:
#번역문 없는 것 지우기 => 105395개의 문단
data = data[data['번역문']!='번역문 오류']
data

,ID,원문,번역문
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)..."
...,...,...,...
105619,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)..."
105620,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋..."
105621,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것..."
105622,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格..."


In [9]:
data[data['원문']== '원문 오류']

,ID,원문,번역문
2526,ITKC_MO_0005A_0040_020_0120,원문 오류,"\n\n칠언고시인데, 여기에는 수록하지 않는다.\n\n"
8764,ITKC_MO_0024A_0160_050_0010,원문 오류,\n백암산 정토사기(白巖山淨土寺記) 없어졌다.\n\n【안】 이색(李穡)의 정토사(淨...
8765,ITKC_MO_0024A_0160_060_0010,원문 오류,\n본조의 칙위성지의 발어 정축 [本朝勅慰聖旨跋語 丁丑] 없어졌다.\n\n
8766,ITKC_MO_0024A_0160_070_0010,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8767,ITKC_MO_0024A_0160_080_0010,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8768,ITKC_MO_0024A_0160_080_0020,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8769,ITKC_MO_0024A_0160_080_0030,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8770,ITKC_MO_0024A_0160_080_0040,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8771,ITKC_MO_0024A_0160_080_0050,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...
8772,ITKC_MO_0024A_0160_080_0060,원문 오류,\n여조의 적경원 중흥비 경오 [麗朝積慶園中興碑 庚午] 없어졌다.\n학자 지남도(學...


### 겹치는 부분 제거

In [10]:
# id 겹치는 것 = 없음
data[data.duplicated(['ID'])==True]

,ID,원문,번역문


In [11]:
#원문 겹치는 것 
data[data.duplicated(['원문'])==True]

,ID,원문,번역문
1615,ITKC_MO_0004A_0340_010_0050,上延昌侯,"\n연창후에게천경(天經 해[日]를 말한다)이 좌로 돌고 삭통(朔統)이 우로 도니, ..."
1683,ITKC_MO_0004A_0360_010_0040,敎書,"\n\n관고\n운운. 대개 지인(至人)의 경지는 사는 것을 즐거움으로 삼지 않고, ..."
1687,ITKC_MO_0004A_0360_010_0080,敎書,"\n\n교서\n운운. 대개 국가가 석문(釋門)의 품계를 만들어 놓은 것은, 그 본의..."
1689,ITKC_MO_0004A_0360_010_0100,敎書,\n\n교서\n운운. 대개 대부가 70세가 되면 법에 의하여 사퇴를 원하니 이것이 ...
1694,ITKC_MO_0004A_0360_010_0150,\n敎書,\n운운. 우리나라에서 척리(戚里)에 대하여 공(公)ㆍ후(侯)ㆍ백(伯) 3등의 작위...
...,...,...,...
102429,ITKC_MO_0650A_0210_010_0350,大老西歸殷室傾。天心不與獨夫橫。採薇叩馬齊東語。誤了淸風百世名。,\n\n대로 서쪽으로 돌아가자 은나라 기울어 / 大老西歸殷室傾천심은 독부의 횡포와 ...
102430,ITKC_MO_0650A_0210_010_0360,白沙浩漫漫。悲風日暮起。借問此何地。云是古易水。昔日燕儲君。送別入秦士。入秦夫如何。一死酬知...,\n흰모래 벌 넓고 아득한데 / 白沙浩漫漫슬픈 바람이 석양에 일어나네 / 悲風日暮起...
103574,ITKC_MO_0663A_0020_010_0390,朝,\n아침〔朝〕꿈에서 깨자 서창에 달 지려는데 / 夢覺西窓月欲低숲에 내리던 비 비로소...
104334,ITKC_MO_0663A_0170_010_0140,\n\n\n\n\n\n삽화 새창열기\n\n,\n\n\n\n\n\n\n삽화 새창열기\n\n\n


In [12]:
#번역문 겹치는 것  => 논의 필요 
data[data.duplicated(['번역문'])==True]

,ID,원문,번역문
2250,ITKC_MO_0004A_0500_010_0320,江天暮雪,\n\n저물녘에 내리는 강천의 눈\n해 저물어 어둡지 않은 곳이 없는데 / 晩來無處...
2789,ITKC_MO_0011A_0060_010_0190,萬尋澄澈靑銅鏡。千尺逶迤白玉虹。怪底古今流不盡。層巖直上是龍宮。,\n\n평사(平沙)에 내리는 기러기\n줄줄이 점점이 가지런했다 비꼈다 / 行行點點整...
2935,ITKC_MO_0011A_0130_030_0020,問。孟子曰。夏后氏五十而貢。殷人七十而助。周人百畝而徹。其實皆什一也。又曰。仁政必自經界始。...,\n묻노라.《논어(論語)》를 읽을 때에는 언제나 여러 제자들이 물은 것을 자신이 직...
2936,ITKC_MO_0011A_0130_030_0030,問。帝王之統。若四時之相代。有不可紊。天命人心之所歸。又不可誣也。三五已前。遐哉邈乎。雖有載...,\n묻노라.《논어(論語)》를 읽을 때에는 언제나 여러 제자들이 물은 것을 자신이 직...
2937,ITKC_MO_0011A_0130_030_0040,問。論語曰。旣庶矣。富之。旣富矣。敎之。又曰。善人敎民七年。可使卽戎。又曰。善人爲邦百年。亦...,\n묻노라.《논어(論語)》를 읽을 때에는 언제나 여러 제자들이 물은 것을 자신이 직...
...,...,...,...
103579,ITKC_MO_0663A_0020_010_0440,擘岸揷靑冥。參天抱雲木。寺樓刱何年。巍然當其谷。傑閣始隱映。長廊乍起伏。燈炯照僧衣。鍾動隨佛...,\n출발을 기록하다〔紀發行〕약관의 나이에 산수 구경 좋아하니 / 弱歲喜幽賞진토는 편...
103580,ITKC_MO_0663A_0020_010_0450,仲秋深谷寒。行子心悄悄。一路稍依微。千山遞遶圍。苦竹不見外。村落時縹緲。其人虎爲羣。其屋日不...,\n출발을 기록하다〔紀發行〕약관의 나이에 산수 구경 좋아하니 / 弱歲喜幽賞진토는 편...
103581,ITKC_MO_0663A_0020_010_0460,\n乾坤闢鴻濛。山嶽氣所凝。茲峰一卷撮。萬里乃相仍。我來値秋半。風高天宇澄。穹林拂蒙鬱。絶壑...,\n출발을 기록하다〔紀發行〕약관의 나이에 산수 구경 좋아하니 / 弱歲喜幽賞진토는 편...
103582,ITKC_MO_0663A_0020_010_0470,昔遊大源寺。愛此無塵雜。吟哦共詩流。選搜携殘衲。徘徊盡淸奇。夢寐思騰踏。今秋興何長。雙屐無餘...,\n출발을 기록하다〔紀發行〕약관의 나이에 산수 구경 좋아하니 / 弱歲喜幽賞진토는 편...


In [13]:
data_dup = data.drop_duplicates(['번역문'])
data_dup

,ID,원문,번역문
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)..."
...,...,...,...
105619,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)..."
105620,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋..."
105621,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것..."
105622,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格..."


In [ ]:
data_dup.to_pickle('한국문집총간_trans_duplicate.pickle')

In [ ]:
print(data.loc[100000]['번역문'])


근래에 모친을 모시는 기후가 다복하시리라 생각합니다. 혼례를 올릴 날이 점점 다가오니, 비할 데 없이 기쁘시겠지만, 혹 정신을 번거롭게 하는 데에 이르지는 않았는지요? 우리가 혼례를 올릴 때에는 근본과 실제를 도탑게 하는 것을 중요하게 여겨야 합니다. 무릇 겉치레에 해당하는 절차는 모두 생략해야 합니다. 이는 《주역》에서 질그릇에 담는 지극한 의미입니다. 그대는 평소에 이치를 강구하였으므로 혹 이미 이 의리를 깨달았을 것입니다. 시집간 다음에 비녀를 꽂는 것은 우리나라의 잘못된 풍속인데 그대가 예법에 의거하여 바로잡아 화양(華陽 송시열) 부자의 가법(家法)을 깊이 얻고자 하니, 매우 좋습니다.그러나 며느리의 비녀와 옷은 시댁에서 마련하는 것이 오래전부터 전해 온 풍습입니다. 지금 비록 친영하기 전이지만 덩그러니 아무 조치도 취하지 않고 있자니 심정이 편안하지 않습니다. 이에 우리 가문에서 옛날부터 해 오던 대로 관자와 배자를 각각 하나씩 만들어 올리니 혼례를 행할 때 써야 할 곳에 쓰십시오. 《가례》 납폐장(納幣章)을 살펴보면, 비녀와 팔찌 등을 보내는 뜻을 허락하였으니, 이는 예법에 아마도 또한 장애가 되지 않을 것입니다. 삼가 살펴보시고 받아 주시기 바랍니다. 이만 줄입니다.



In [ ]:
re.sub('[一-龥]', '', data.loc[100000]['번역문'])

'\n근래에 모친을 모시는 기후가 다복하시리라 생각합니다. 혼례를 올릴 날이 점점 다가오니, 비할 데 없이 기쁘시겠지만, 혹 정신을 번거롭게 하는 데에 이르지는 않았는지요? 우리가 혼례를 올릴 때에는 근본과 실제를 도탑게 하는 것을 중요하게 여겨야 합니다. 무릇 겉치레에 해당하는 절차는 모두 생략해야 합니다. 이는 《주역》에서 질그릇에 담는 지극한 의미입니다. 그대는 평소에 이치를 강구하였으므로 혹 이미 이 의리를 깨달았을 것입니다. 시집간 다음에 비녀를 꽂는 것은 우리나라의 잘못된 풍속인데 그대가 예법에 의거하여 바로잡아 화양( 송시열) 부자의 가법()을 깊이 얻고자 하니, 매우 좋습니다.그러나 며느리의 비녀와 옷은 시댁에서 마련하는 것이 오래전부터 전해 온 풍습입니다. 지금 비록 친영하기 전이지만 덩그러니 아무 조치도 취하지 않고 있자니 심정이 편안하지 않습니다. 이에 우리 가문에서 옛날부터 해 오던 대로 관자와 배자를 각각 하나씩 만들어 올리니 혼례를 행할 때 써야 할 곳에 쓰십시오. 《가례》 납폐장()을 살펴보면, 비녀와 팔찌 등을 보내는 뜻을 허락하였으니, 이는 예법에 아마도 또한 장애가 되지 않을 것입니다. 삼가 살펴보시고 받아 주시기 바랍니다. 이만 줄입니다.\n'

### 번역문 한자 및 특수문자 제거

In [14]:
# 전처리 함수화
def preprocess(title,text):
    text = re.sub(f'\n{title}\n\n<sep>','',text)
    text = re.sub(f'^\n{title}\n\n','',text)
    text = re.sub('<sep>\n\n$','',text)
    text = re.sub('\n',' ',text)
    text = re.sub('\=+ \d{1,} \=+(\<sep\>)*','',text)
    
    #한자제거 추가
    text = re.sub(r'\([\W]*\)', '', text)
    text = re.sub('[一-龥]', '', text)
    text = re.sub("[()]","", text)
    
    text = regex.sub('\=+ \p{Hangul}+ \=+(\<sep\>)*','',text)
    text = regex.sub('\=+ \p{Han}+ \=+(\<sep\>)*','',text)
    text = text.strip()
    text = re.sub('<sep>\n*$','',text)
    text_list = text.split('<sep>')
    text_list = [x.strip() for x in text_list if not re.match('\=+.+\=+',x)]
    # text_list = [x for x in text_list if not re.match('\=+.+\=+',x)]
    return text_list

In [15]:
data_dup=data_dup.reset_index(drop=True)

In [16]:
tqdm.pandas()

In [17]:
data_dup['text_list'] = data_dup.progress_apply(lambda x:preprocess(x['ID'],x['번역문']),axis=1)
data_dup['text_list']

100%|██████████| 104127/104127 [00:46<00:00, 2256.01it/s]


0         [《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 ...
1         [《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나...
2         [회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시...
3         [신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살펴...
4         [신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 돌...
                                ...                        
104122    [공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 휘...
104123    [부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 휘...
104124    [이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 6...
104125    [글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을...
104126    [오호라, 이 책은 우리 향산공의 유문이다. 공의 큰 절의가 이미 하늘에 있는 해와...
Name: text_list, Length: 104127, dtype: object

In [18]:
data_dup

,ID,원문,번역문,text_list
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,[《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 ...
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,[《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로나...
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","[회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 시..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,[신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살펴...
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","[신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 돌..."
...,...,...,...,...
104122,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","[공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 휘..."
104123,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","[부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 휘..."
104124,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","[이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 6..."
104125,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","[글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글을..."


In [ ]:
data_dup['text_list'][104124]

In [ ]:
data_dup['번역문'][104124]

In [19]:
data_dup['text_list'] = data_dup['text_list'].astype(str)

In [20]:
data_dup.to_pickle('한국문집총간_preprocessed.pickle')

##### 문단 -> 문장

In [27]:
data_dup

,ID,원문,번역문,text_list
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ..."
...,...,...,...,...
104122,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ..."
104123,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ..."
104124,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ..."
104125,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글..."


In [30]:
data_dup['text_list'][0]

"['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리에 놓는 것은 물이 모든 맛의 근본임을 중시하려고 해서이다. 각종 화려한 무늬의 옷감이 아름답기는 하지만 제사 때에 거친 삼베로 동이를 덮는 것은 부녀자들이 하는 길쌈질의 시초를 돌아보고 귀하게 여기려 해서이다.〔     〕”라고 하였다. 옛날의 군자가 그 근본과 시초를 반드시 중하게 여기려 한 것이 이와 같았다.우리 동방에 문장이 나와서 글을 지어 후세에 전할 수 있게 된 것은 고운 최공으로부터 비롯되었다. 그리고 우리 동방의 선비로서 북쪽으로 중국에 유학하여 문장으로 천하에 명성을 떨친 것도 최공으로부터 시작되었다. 최공의 글 가운데 후세에 전하는 것으로는 오직 《계원필경》과 《중산복궤집》 2부가 있을 뿐이다. 그러고 보면 이 2부의 서책이 또한 우리 동방 문장의 근본이요 시초라고 할 것이다.글을 숭상하는 우리 동방의 풍조가 아조에 이르러서는 더욱 빛나고 무르익어 집집마다 연허와 조유가 배출되면서 시와 문으로 문집을 이룬 것이 집을 가득 채울 정도로 엄청나게 많았다. 하지만 최공의 글을 아는 사람은 드물기만 하였다.내가 일찍이 근대 사람이 편찬한 《동국서목》을 보니 《중산복궤집》이 실려 있기에 널리 구해 보았지만 끝내 얻지 못하였다. 다만 《계원필경》 20권은 우리 집안에서 대대로 소장해 왔으므로 내가 어려서부터 진귀한 글로 알고서 음미해 왔다. 그러나 간혹 사람들과 이야기를 나누다 보면, 비록 박아하고 글을 잘하며 옛것을 좋아하는 자라고 할지라도 모두 그 글을 본 적이 없다고 말하고 있으니, 그러고 보면 이 글이 거의 없어질 지경에 이르렀다고 해야 할 것이다. 만약 이 글이 동방에 전하지 않게 된다면, 이는 현주를 태실에 진설하지 않는 것과 같고, 거친 삼베로 제사용 술동이를 덮지 않는 것과 같게 될 것이니, 어떻게 백성에게 근본을 잊지 말라고 가르칠 수가 있겠는가.세상에서는 간혹 공의 글이 모두 변려 사륙문으로서 옛 작자의 글과는 완전히 다르다고 비평하기도 한다.

In [31]:
split_sentences(data_dup['text_list'][0])

[Korean Sentence Splitter]: Initializing Pynori...


["['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을 윗자리에 놓는 것은 물이 모든 맛의 근본임을 중시하려고 해서이다.",
 '각종 화려한 무늬의 옷감이 아름답기는 하지만 제사 때에 거친 삼베로 동이를 덮는 것은 부녀자들이 하는 길쌈질의 시초를 돌아보고 귀하게 여기려 해서이다.〔 〕”라고 하였다.',
 '옛날의 군자가 그 근본과 시초를 반드시 중하게 여기려 한 것이 이와 같았다.',
 '우리 동방에 문장이 나와서 글을 지어 후세에 전할 수 있게 된 것은 고운 최공으로부터 비롯되었다.',
 '그리고 우리 동방의 선비로서 북쪽으로 중국에 유학하여 문장으로 천하에 명성을 떨친 것도 최공으로부터 시작되었다.',
 '최공의 글 가운데 후세에 전하는 것으로는 오직 《계원필경》과 《중산복궤집》 2부가 있을 뿐이다.',
 '그러고 보면 이 2부의 서책이 또한 우리 동방 문장의 근본이요 시초라고 할 것이다.',
 '글을 숭상하는 우리 동방의 풍조가 아조에 이르러서는 더욱 빛나고 무르익어 집집마다 연허와 조유가 배출되면서 시와 문으로 문집을 이룬 것이 집을 가득 채울 정도로 엄청나게 많았다. 하지만 최공의 글을 아는 사람은 드물기만 하였다.',
 '내가 일찍이 근대 사람이 편찬한 《동국서목》을 보니 《중산복궤집》이 실려 있기에 널리 구해 보았지만 끝내 얻지 못하였다.',
 '다만 《계원필경》 20권은 우리 집안에서 대대로 소장해 왔으므로 내가 어려서부터 진귀한 글로 알고서 음미해 왔다.',
 '그러나 간혹 사람들과 이야기를 나누다 보면, 비록 박아하고 글을 잘하며 옛것을 좋아하는 자라고 할지라도 모두 그 글을 본 적이 없다고 말하고 있으니, 그러고 보면 이 글이 거의 없어질 지경에 이르렀다고 해야 할 것이다.',
 '만약 이 글이 동방에 전하지 않게 된다면, 이는 현주를 태실에 진설하지 않는 것과 같고, 거친 삼베로 제사용 술동이를 덮지 않는 것과 같게 될 것이니, 어떻게 백성에게 근본을 잊지 말라고 가르칠 수가 있겠는가.',
 '세상에서는 간

In [ ]:
help(split_sentences)


Help on function split_sentences in module kss.kss:

split_sentences(text: Union[str, tuple, List[str]], use_heuristic: bool = True, use_quotes_brackets_processing: bool = False, max_recover_step: int = 5, max_recover_length: int = 20000, backend: str = 'pynori', num_workers: int = -1, disable_gc: bool = True) -> Union[List[str], List[List[str]]]
    Split document to sentences.
    
    Args:
        text (Union[str, tuple, List[str]]): input text
        use_heuristic (bool): use heuristic algorithms or not
        use_quotes_brackets_processing (bool): use quotes or bracket processing or not
        max_recover_step (int): maximum step for quote and bracket misalignment recovering
        max_recover_length (int): maximum text length to recover when quote and bracket misaligned
        backend (str): max length of text to use morpheme feature
        num_workers (int): number of multiprocessing workers ('-1' means maximum processes)
        disable_gc (bool): disable garbage collect

In [32]:
# 문단 -> 문장 (오래걸림)
data_dup['sentence'] = data_dup['text_list'].progress_map(lambda x:[split_sentences(w) for w in x])
data_dup['sentence']

  0%|          | 0/104127 [00:00<?, ?it/s]Process ForkProcess-1267:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
Exception ignored in: <function _releaseLock at 0x7f147565f8c0>
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "

KeyboardInterrupt: ignored

In [ ]:
# flatten list
df['sentence'] = df['sentence'].progress_map(lambda x:(list(itertools.chain(*x))))
df['sentence']

0       [백범일지  저자의 말. 하여서 딱 거절하였다., 그런즉 일인이 다시 말하기를,, ...
1       [날개, ‘박제가 되어 버린 천재’를 아시오?, 나는 유쾌하오., 이런 때 연애까지...
2       [산골 나그네, 밤이 깊어도 술군은 역시들지 않는다., 메주 뜨는 냄새와 같이 쾨쾨...
3       [달을 쏘다, 번거롭던 사위(四圍)가 잠잠해지고 시계 소리가 또렷하나 보니 밤은 저...
4       [지팽이 역사, 아침에 깨이기는 일찍 깨었다는 증거로 닭 우는 소리를 들었는데 또 ...
                              ...                        
998     [대행애사, 병인사월이십오일(丙寅四月二十五日)에 대행(大行)이 등하(登遐)하시다.,...
999     [이충무공 묘산경매문제  일. 환구오주(寰球五洲)에 어느 땅을 무를 것 없이 그 민...
1000    [가을과 산양, 화단 위 해바라기 송이가 칙칙하게 시들었을 젠 벌써 가을이 완연한듯...
1001    [가로의 요술사, “자 똑똑히 들어보세요, 똑똑히.”, 다 낡아서 구리쇠빛으로 변한...
1002    [가지말게, 다 찌그러져가는 우막집!, 이 까짓 것을 누가 단 일 원이라도 내고 사...
Name: sentence, Length: 969, dtype: object

In [ ]:
# 학습용 문장 형태로 전환
df_sent = pd.DataFrame()
for idx in tqdm(df.index):
    title_wiki = df['doc|@title'][idx]
    title=df['작품명'][idx]
    period = df['시대_N'][idx]
    res = pd.DataFrame(df['sentence'][idx],columns=['sentence'])
    res.insert(0,'title_wiki',title_wiki)
    res.insert(1,'title',title)
    res.insert(3,'period',period)
    df_sent = df_sent.append(res)

In [ ]:
df_sent # 30만개가 넘는 문장 확보

,title_wiki,title,sentence,period
0,백범일지,백범일지,백범일지 저자의 말. 하여서 딱 거절하였다.,E3
1,백범일지,백범일지,"그런즉 일인이 다시 말하기를,",E3
2,백범일지,백범일지,"""형법이 곧 대군주 폐하의 명령이 아니오? 그런즉 김창수를 수갑을 채우고 포승으로 ...",E3
3,백범일지,백범일지,하고 기어이 나를 결박하여놓고 사진박기를 주장하였다.,E3
4,백범일지,백범일지,"이에 김윤정은,",E3
...,...,...,...,...
34,가지말게,가지말게,“아이고 이 사람아 그만 가지 말게 저 어린 것들도 무엇을 알고 울겠나.”,E2
35,가지말게,가지말게,하는 소리가 나자 남 먼저 갑동이와 순삼의 얼굴이 벙싯 웃는 듯 경련을 짓더니,E2
36,가지말게,가지말게,“어허이고 -.”,E2
37,가지말게,가지말게,하고 소리를 내자 온 방 안은 ‘왕 -’ 울음 소리에 차고 말았다.,E2


In [ ]:
len(df_sent['title'].unique())

586

In [ ]:
df_sent['period'].value_counts().sort_index()

E1     31311
E2    260145
E3     53344
Name: period, dtype: int64

### 형태소 분석


In [23]:
## 토크나이징 함수 선언 (품사 포함)
stopwords = ['이다','하다','있다','위하']
def tokenize(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S') and word not in stopwords] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거

In [24]:
kiwi.analyze('우리는 어디로 가야합니까?')

[([Token(form='우리', tag='NP', start=0, len=2),
   Token(form='는', tag='JX', start=2, len=1),
   Token(form='어디', tag='NP', start=4, len=2),
   Token(form='로', tag='JKB', start=6, len=1),
   Token(form='가', tag='VV', start=8, len=1),
   Token(form='야', tag='EC', start=9, len=1),
   Token(form='하', tag='VX', start=10, len=1),
   Token(form='ᆸ니까', tag='EF', start=11, len=2),
   Token(form='?', tag='SF', start=13, len=1)],
  -35.57164001464844)]

In [25]:
kiwi.analyze('대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다')

[([Token(form='대한민국', tag='NNP', start=0, len=4),
   Token(form='의', tag='JKG', start=4, len=1),
   Token(form='주권', tag='NNG', start=6, len=2),
   Token(form='은', tag='JX', start=8, len=1),
   Token(form='국민', tag='NNG', start=10, len=2),
   Token(form='에게', tag='JKB', start=12, len=2),
   Token(form='있', tag='VV', start=15, len=1),
   Token(form='고', tag='EC', start=16, len=1),
   Token(form=',', tag='SP', start=17, len=1),
   Token(form='모든', tag='MM', start=19, len=2),
   Token(form='권력', tag='NNG', start=22, len=2),
   Token(form='은', tag='JX', start=24, len=1),
   Token(form='국민', tag='NNG', start=26, len=2),
   Token(form='으로부터', tag='JKB', start=28, len=4),
   Token(form='나오', tag='VV', start=33, len=2),
   Token(form='ᆫ다', tag='EC', start=35, len=1)],
  -78.57364654541016)]

In [33]:
# 조사, 어미, 특수기호 제외한 토크나이징
data_dup['token'] = data_dup['text_list'].progress_map(lambda x:tokenize(x))

100%|██████████| 104127/104127 [1:49:07<00:00, 15.90it/s]


In [34]:
#토큰 2개 이하인 것 확인
data_dup[data_dup['token'].map(lambda x:len(x)<=2)]

,ID,원문,번역문,text_list,token
3310,ITKC_MO_0014A_0120_010_0040,[本文缺],\n\n본문 결락\n\n,['본문 결락'],"[본문/NNG, 결락/NNG]"
21305,ITKC_MO_0157A_0210_000_0070,[本文缺],\n,[''],[]
41777,ITKC_MO_0286A_0650_010_0110,乾目太陽巽色少剛乾性太陽巽木少剛,\n건목태양(乾目太陽)태이태음(兌耳太陰)이비소양(離鼻少陽)진구소음(震口少陰)손색소강...,['건목태양태이태음이비소양진구소음손색소강감성소유간기태강곤미태유건성태양태정태음이형소양...,[건목태양태이태음이비소양진구소음손색소강감성소유간기태강곤미태유건성태양태정태음이형소양진...
51282,ITKC_MO_0332A_0070_070_0010,오류,\n\n- 본문결(本文缺) -\n\n,['- 본문결 -'],"[본문/NNG, 결/NNG]"
70841,ITKC_MO_0433A_0130_010_0010,天命之謂性。率性之謂道。修道之謂敎。道也者。不可須臾離也。可離非道也。是故君子。戒愼乎其所不...,"\n天命之謂性, 率性之謂道, 修道之謂敎. 道也者, 不可須臾離也. 可離, 非道也. 是...","[', , . , . , . , , . , . . , ; , . , . , . ,...",[]
82625,ITKC_MO_0560A_0090_010_0020,九九數 呼小數在上。大數在下。\n,"\n9×9―81, 8×9―72, 7×9―63, 6×9―54, 5×9―45, 4×9―...","['9×9―81, 8×9―72, 7×9―63, 6×9―54, 5×9―45, 4×9―...",[]


In [35]:
#토큰 2개 이하인 것 확인
df_token = data_dup[data_dup['token'].map(lambda x:len(x)>2)]

In [36]:
df_token = df_token.dropna(axis=0)

In [38]:
df_token

,ID,원문,번역문,text_list,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,"[예기/NNP, 이르다/VV, 달다/VA, 술/NNG, 맛/NNG, 좋다/VA, 하..."
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,"[계원필경/NNP, 집/NNG, 권/NNB, 신라/NNP, 고운/NNP, 최/NNP..."
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...","[회남/NNP, 본국/NNG, 들어오다/VV, 조서/NNG, 등/NNB, 보내다/V..."
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,"[신/NNP, 모/NP, 아뢰다/VV, 금/NNG, 월/NNB, 모이다/VV, 진주..."
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...","[신/NNP, 모/NP, 아뢰다/VV, 신/NNG, 진주/NNP, 원/NNG, 장보..."
...,...,...,...,...,...
104122,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...","[공/NNG, 휘다/VV, 만도/NNP, 자/NNP, 관필/NNG, 이다/VCP, ..."
104123,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...","[부군/NNG, 휘다/VV, 만도/NNP, 자/NNP, 관/NNG, 필/NNB, 호..."
104124,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...","[이것/NP, 우리/NP, 향산공/NNG, 수고/NNG, 인출/NNG, 편/NNB,..."
104125,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...","[글/NNG, 기/XSN, 드러나다/VV, 것/NNB, 이다/VCP, 기/NNG, ..."


In [37]:
df_token.to_pickle('한국문집총간_token.pickle')

In [39]:
# 상위 n개 단어 확인
cnt = Counter(list(itertools.chain(*df_token['token'].tolist())))
cnt.most_common(30) # 상위 N개

NameError: ignored

In [ ]:
with open('df_par.pkl','wb') as f:
    pickle.dump(df_par,f)

In [ ]:
df_pa =pd.read_pickle('df_par.pkl')

### 원문과 번역문에서 우리나라 표현 개수 세기

In [ ]:
data_dup[data_dup['text_list'].str.contains('우리나라')]

,ID,원문,번역문,text_list
147,ITKC_MO_0001A_0120_020_0050,昨奉公牒。云當道臨淮叛卒。過江來投。集衆廣場。已受降訖。三復來示。言難不酬。且降之爲名。其理...,"\n전일에 공첩(公牒)을 받아 보건대, “당도(當道)는 임회(臨淮)의 반졸(叛卒)이...","['전일에 공첩을 받아 보건대, “당도는 임회의 반졸이 강을 건너 투항해 왔기에, ..."
352,ITKC_MO_0002A_0030_010_0180,靑鶴洞碑。鄭弘溟銘曰。若高麗，百濟，新羅。國雖一域。粤有蓬萊，瀛洲，方丈。山則三神。積氣扶桑...,"\n고려와 백제와 신라로 말하면, 나라는 비록 한 지역에 속한다고 하겠지만 봉래(蓬...","['고려와 백제와 신라로 말하면, 나라는 비록 한 지역에 속한다고 하겠지만 봉래와 ..."
390,ITKC_MO_0002A_0040_030_0020,臣某言。臣聞欲而不貪。駕說於孔門弟子。德莫若讓。騰規於晉國行人。苟竊位自安。則妨賢是責。臣假...,"\n신 모는 아룁니다. 신이 듣건대, “원하되 욕심부리지 않는다.〔欲而不貪)”라는 ...","['신 모는 아룁니다. 신이 듣건대, “원하되 욕심부리지 않는다.〔”라는 말은 공문..."
392,ITKC_MO_0002A_0040_030_0040,臣某言。前權知當國王事臣坦是親叔。臣自亡父贈大傅臣晸及次叔臣晃。相次亡沒。叔權守蕃服。疾故相...,\n신 모는 아룁니다. 본국의 왕사(王事)를 전에 임시로 맡았던 신(臣) 탄(坦)은...,['신 모는 아룁니다. 본국의 왕사를 전에 임시로 맡았던 신 탄은 신의 친고모입니다...
407,ITKC_MO_0002A_0050_010_0010,帝唐揃亂以武功。乾符戊戌。滅黃巢。 易元以文德之年。暢月。仲冬曰暢月 月缺之七日。日蘸咸池時...,\n\n제당(帝唐)이 무공으로 난리를 평정하고 문덕(文德)으로 개원(改元)한 해의 ...,"['제당이 무공으로 난리를 평정하고 문덕으로 개원한 해의 창월, 달이 이지러지기 시..."
...,...,...,...,...
105372,ITKC_MO_1252A_0250_010_0050,公諱而楨字公直。李氏以高麗碧珍將軍諱忩言爲始祖。中世諱堅幹進賢舘大提學號山花先生。自是世襲華...,"\n공은, 휘는 이정(而楨), 자는 공직(公直)이다. 이씨(李氏)는 고려 벽진장군(...","['공은, 휘는 이정, 자는 공직이다. 이씨는 고려 벽진장군 휘 총언이 시조이다. ..."
105386,ITKC_MO_1252A_0260_000_0010,天降忠賢。多在末造。欲其擔綱常也。我邦歷五百年而天命訖焉。時則響山李先生殉之。其大道卓節磊落...,"\n하늘이 충현(忠賢)을 내려 주는 것은 나라의 말기일 때가 많으니, 이는 그 사람...","['하늘이 충현을 내려 주는 것은 나라의 말기일 때가 많으니, 이는 그 사람이 강상..."
105387,ITKC_MO_1252A_0270_000_0010,公諱晩燾字觀必號響山。又曰直齋。姓李氏眞城人。麗末以縣吏中司馬諱碩其始祖。六傳而至文純公諱滉...,"\n공은, 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 또는 직재(直齋...","['공은, 휘는 만도, 자는 관필, 호는 향산 또는 직재이다. 성은 이씨, 본관은 ..."
105389,ITKC_MO_1252A_0270_000_0030,惟我國家隆盛。退陶夫子以道學啓一治文明之運。國家將終。響山先生以節義扶萬世綱常之重。盖道學節...,\n우리나라가 융성하였을 때 퇴도(退陶) 선생이 도학으로써 일치문명(一治文明)의 운...,"['우리나라가 융성하였을 때 퇴도 선생이 도학으로써 일치문명의 운세를 열었고, 나라..."


In [ ]:
#열 추가
data_dup['우리나라'] =data_dup['text_list'].str.count('우리나라')
data_dup

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,원문,번역문,text_list,우리나라
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,0
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,0
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",0
...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",4
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",0
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",0


In [ ]:
#우리 나라의 한글 개념어 숫자 세기
ko_list = ['우리', '우리나라', '우리 나라']
for idx in ko_list:
  data_dup[idx] =data_dup['text_list'].str.count(idx)

data_dup

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ID,원문,번역문,text_list,우리나라,우리,우리 나라
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,0,5,0
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,0,1,0
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,0,1,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",0,2,0
...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,0
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",4,37,0
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",0,2,0
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",0,1,0


In [ ]:
#우리나라의 한자 개념어 숫자세기
ch_list = ['國' ,'我國' ,'東國', '海東' ,'三韓' ,'朝鮮' ,'我朝']
for idx in ch_list:
  data_dup[idx] =data_dup['원문'].str.count(idx)

data_dup

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ID,원문,번역문,text_list,우리나라,우리,우리 나라,國,我國,東國,海東,三韓,朝鮮,我朝
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,0,5,0,8,0,3,0,0,0,1
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,0,1,0,2,0,0,0,0,0,0
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,1,0,0,0,0,0,0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,0,1,0,0,0,0,0,0,0,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",0,2,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,0,4,0,0,0,0,0,0
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",4,37,0,53,2,0,0,0,2,0
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",0,2,0,2,0,0,0,0,0,0
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",0,1,0,1,0,0,0,0,0,0


In [ ]:
#한국어 우리나라 합치기
data_dup['Korean'] = data_dup['우리']+data_dup['우리나라']+data_dup['우리 나라']
data_dup

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,원문,번역문,text_list,우리나라,우리,우리 나라,國,我國,東國,海東,三韓,朝鮮,我朝,Korean
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,0,5,0,8,0,3,0,0,0,1,5
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,0,1,0,2,0,0,0,0,0,0,1
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,1,0,0,0,0,0,0,0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,0,1,0,0,0,0,0,0,0,0,1
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",0,2,0,1,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,0,4,0,0,0,0,0,0,0
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",4,37,0,53,2,0,0,0,2,0,41
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",0,2,0,2,0,0,0,0,0,0,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",0,1,0,1,0,0,0,0,0,0,1


In [ ]:
#한자 우리나라 합치기
data_dup['Chinese'] = data_dup['國']+data_dup['我國']+data_dup['東國']+data_dup['海東']+data_dup['三韓']+data_dup['朝鮮']+data_dup['我朝']
data_dup

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,원문,번역문,text_list,우리나라,우리,우리 나라,國,我國,東國,海東,三韓,朝鮮,我朝,Korean,Chinese
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,0,5,0,8,0,3,0,0,0,1,5,12
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,0,1,0,2,0,0,0,0,0,0,1,2
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,1,0,0,0,0,0,0,0,1
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,0,1,0,0,0,0,0,0,0,0,1,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",0,2,0,1,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,0,4,0,0,0,0,0,0,0,4
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",4,37,0,53,2,0,0,0,2,0,41,57
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",0,2,0,2,0,0,0,0,0,0,2,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",0,1,0,1,0,0,0,0,0,0,1,1


In [ ]:
data_guk = data_dup
data_guk

,ID,원문,번역문,text_list,우리나라,우리,우리 나라,國,我國,東國,海東,三韓,朝鮮,我朝,Korean,Chinese
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,0,5,0,8,0,3,0,0,0,1,5,12
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,0,1,0,2,0,0,0,0,0,0,1,2
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,1,0,0,0,0,0,0,0,1
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,0,1,0,0,0,0,0,0,0,0,1,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",0,2,0,1,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,0,4,0,0,0,0,0,0,0,4
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",4,37,0,53,2,0,0,0,2,0,41,57
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",0,2,0,2,0,0,0,0,0,0,2,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",0,1,0,1,0,0,0,0,0,0,1,1


In [ ]:
#데이터를 줄이기 위해 한 열 삭제
data_guk_1 = data_guk.drop('번역문', axis=1)

In [ ]:
data_guk_1.rename(columns = {"text_list": "번역문"}, inplace = True)
data_guk_1

,ID,원문,번역문,우리나라,우리,우리 나라,國,我國,東國,海東,三韓,朝鮮,我朝,Korean,Chinese
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,0,5,0,8,0,3,0,0,0,1,5,12
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,0,1,0,2,0,0,0,0,0,0,1,2
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,1,0,0,0,0,0,0,0,1
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,0,1,0,0,0,0,0,0,0,0,1,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",0,2,0,1,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,0,4,0,0,0,0,0,0,0,4
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",4,37,0,53,2,0,0,0,2,0,41,57
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",0,2,0,2,0,0,0,0,0,0,2,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",0,1,0,1,0,0,0,0,0,0,1,1


### 원문과 번역문에서 우리 표현 개수 세기

In [ ]:
data_dup

In [ ]:
#'우리'의 한글 개념어 숫자 세기
data_dup['우리'] =data_dup['text_list'].str.count('우리')

data_dup

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,원문,번역문,text_list,우리
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,5
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,1
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,1
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",2
...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",37
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",1


In [ ]:
#우리나라의 한자 개념어 숫자세기
ch_list = ['我' , '吾']
for idx in ch_list:
  data_dup[idx] =data_dup['원문'].str.count(idx)

data_dup

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID,원문,번역문,text_list,우리,我,吾
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,5,1,5
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,1,1,0
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,1,0,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",2,0,0
...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,1
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",37,43,55
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",2,0,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",1,1,0


In [ ]:
#한자  합치기
data_dup['Chinese'] = data_dup['我']+data_dup['吾']
data_dup

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,원문,번역문,text_list,우리,我,吾,Chinese
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,5,1,5,6
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,1,1,0,1
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,1,0,0,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",2,0,0,0
...,...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,1,1
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",37,43,55,98
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",2,0,2,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",1,1,0,1


In [ ]:
data_uri = data_dup
data_uri

,ID,원문,번역문,text_list,우리,我,吾,Chinese
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,5,1,5,6
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,\n《계원필경집》 20권은 신라의 고운(孤雲) 최공(崔公)이 당나라 회남(淮南) 막...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,1,1,0,1
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"\n회남(淮南)에서 본국에 들어오면서 조서(詔書) 등을 보내는 사신을 겸한, 전(前...","['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,\n신(臣) 모(某)는 아룁니다.금월(今月) 모일(某日)에 진주원(進奏院)의 장보(...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,1,0,0,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"\n신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사(入南蠻通和使)...","['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",2,0,0,0
...,...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"\n공은 휘는 만도(晩燾), 자는 관필(觀必)이며, 성은 이씨(李氏)로 퇴계(退溪)...","['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,1,1
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...","['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",37,43,55,98
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"\n이것은 우리 향산공(響山公)의 수고(手藁)로, 《인출편(訒出編)》이라고 하는 것...","['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",2,0,2,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"\n글이란 기(氣)가 드러난 것이며, 기가 충만(充滿)한 뒤에라야 글 역시 격조(格...","['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",1,1,0,1


In [ ]:
data_uri_1 =data_uri.drop('번역문', axis=1)
data_uri_1

,ID,원문,text_list,우리,我,吾,Chinese
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,5,1,5,6
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,1,1,0,1
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,1,0,0,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",2,0,0,0
...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,1,1
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",37,43,55,98
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",2,0,2,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",1,1,0,1


In [ ]:
data_uri_1.rename(columns = {"text_list": "번역문"}, inplace = True)
data_uri_1

,ID,원문,번역문,우리,我,吾,Chinese
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,['《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,5,1,5,6
1,ITKC_MO_0001A_0010_010_0020,桂苑筆耕集二十卷。新羅孤雲崔公在唐淮南幕府時公私應酬之作。而東還之後。手編表進于朝者也公名致...,['《계원필경집》 20권은 신라의 고운 최공이 당나라 회남 막부에 있을 때 공적으로...,1,1,0,1
2,ITKC_MO_0001A_0010_010_0030,淮南入本國兼送詔書等使前都統巡官承務郎侍御史內供奉賜紫金魚袋臣崔致遠進所著雜詩賦及表奏集二十...,"['회남에서 본국에 들어오면서 조서 등을 보내는 사신을 겸한, 전 도통순관 승무랑 ...",0,0,0,0
3,ITKC_MO_0001A_0020_010_0010,臣某言。今月某日。得進奏院狀報。奉十一日宣下改廣明元年爲中和元年者。展義龜城。易名鳳紀。美號...,['신 모는 아룁니다.금월 모일에 진주원의 장보를 얻어서 11일에 선하된 내용을 살...,1,0,0,0
4,ITKC_MO_0001A_0020_010_0020,臣某言。臣得進奏院狀報。入南蠻通和使劉光裕等廻。雲南通和兼進獻國信金銀器物疋段香藥馬等者。天...,"['신 모는 아룁니다.신이 진주원의 장보를 얻어 보건대, 입남만통화사 유광유 등이 ...",2,0,0,0
...,...,...,...,...,...,...,...
105390,ITKC_MO_1252A_0270_000_0040,公諱晩燾字觀必姓李。退溪文純公后。大司成彙濬娶弼善姜橒女无育。繼娶僉樞宋在憲女。憲宗壬寅生公...,"['공은 휘는 만도, 자는 관필이며, 성은 이씨로 퇴계 문순공의 후손이다. 대사성 ...",0,0,1,1
105391,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"['부군은 휘는 만도, 자는 관필, 호는 향산 혹은 직제이며 성은 이씨이다. 시조 ...",37,43,55,98
105392,ITKC_MO_1252A_0290_000_0010,此吾響山公手藳之名訒出編者也。殉國之粤六年丙辰。吾黨之士。就其中節略爲十九卷原集。又以門生子...,"['이것은 우리 향산공의 수고로, 《인출편》이라고 하는 것이다. 공이 순국하신 지 ...",2,0,2,2
105393,ITKC_MO_1252A_0290_000_0020,文者氣之所形也。氣必充積而後文亦有格力。故觀於文而其人槩可想矣。然氣不可力以致。其得於天者。...,"['글이란 기가 드러난 것이며, 기가 충만한 뒤에라야 글 역시 격조가 생기므로, 글...",1,1,0,1


### 피클 파일과 csv 파일로 만들기

In [ ]:
#우리나라 csv 파일
data_guk_1.to_csv('/content/drive/Shareddrives/ProjectOuri/DB&Code/한국고전종합DB/한국문집총간/번역/한국문집총간_우리나라.csv', encoding='utf-8-sig')

In [ ]:
#우리나라 pickle 파일
data_guk_1.to_pickle('/content/drive/Shareddrives/ProjectOuri/DB&Code/한국고전종합DB/한국문집총간/번역/한국문집총간_우리나라.pickle')

In [ ]:
#'우리' csv 파일
data_uri_1.to_csv('/content/drive/Shareddrives/ProjectOuri/DB&Code/한국고전종합DB/한국문집총간/번역/한국문집총간_우리.csv', encoding='utf-8-sig')

In [ ]:
#'우리' pickle 파일
data_uri_1.to_pickle('/content/drive/Shareddrives/ProjectOuri/DB&Code/한국고전종합DB/한국문집총간/번역/한국문집총간_우리.pickle')